In [16]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time
import json

# 댓글의 정보를 가져오는 메소드
def comments_info(url):
    browser="C:\\Users\CPB06GameN\Anaconda3\envs\chromedriver.exe"
    driver=webdriver.Chrome(browser)
    driver.get(url)
    time.sleep(2)
    
    # 스크롤바를 끝까지 내리기
    body = driver.find_element_by_tag_name("body")
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    # 스크롤바의 길이가 같아질때 까지 반복문
    while True:
        for i in range(0,10):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.1)
        
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        # 스크롤바 길이가 같아지면 반복문을 멈춤
        if new_height == last_height:
            break
        last_height = new_height
    
    
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    reviews = soup.find_all('ytd-comment-renderer')
    comments = []
    
    # 댓글 정보 가져오기
    for review in reviews:
        a ={}
        # 댓글의 등록일
        a['publish_at'] = review.find_all('a',attrs= {'class':'yt-simple-endpoint style-scope yt-formatted-string'})[0].get_text()
        # 댓글의 내용
        a['content'] = review.find_all('yt-formatted-string', attrs= {'id':'content-text'})[0].get_text()
        # 댓글의 좋아요 개수
        a['like_count'] = review.find_all('span', attrs= {'id':'vote-count-middle'})[0].get_text().replace('\n','').replace(' ','')
        comments.append(a)
    driver.close()
    
    #댓글 정보 리턴
    return comments

# 비디오 정보를 수집하는 메소드
def video_info(search,count=10):
    info = []
    # 지정한 count만큼 반복문 진행
    for i in range(1,count+1):
        browser="C:\\Users\CPB06GameN\Anaconda3\envs\chromedriver.exe"
        driver=webdriver.Chrome(browser)
        driver.get("https://www.youtube.com/results?search_query="+search)
        
        try:
            url = driver.find_element_by_css_selector('#contents > ytd-video-renderer:nth-child('+str(i)+') > div > ytd-thumbnail > a').get_attribute('href')
            driver.get(url)
            time.sleep(2)
    
            video_info = {}
            # 설명의 더보기 버튼 클릭
            driver.find_element_by_css_selector('#more > yt-formatted-string').click()
            # 비디오의 제목
            video_info['title'] = driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text
            # 비디오의 설명
            video_info['description'] = driver.find_element_by_css_selector('#description > yt-formatted-string').text
            # 비디오의 등록일
            video_info['publish_at'] = driver.find_element_by_css_selector('#upload-info > span').text
            # 비디오의 조회수
            video_info['view_count'] = driver.find_element_by_css_selector('#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer').text
            # 비디오의 등록아이디
            video_info['videoid'] = url[32:]
            # 비디오의 좋아요 개수
            video_info['like_count'] = driver.find_element_by_css_selector('#top-level-buttons > ytd-toggle-button-renderer:nth-child(1) > a').text
            # 비디오의 싫어요 개수
            video_info['dislike_count'] = driver.find_element_by_css_selector('#top-level-buttons > ytd-toggle-button-renderer:nth-child(2) > a').text
            # 비디오의 카테고리
            video_info['category_id'] = driver.find_element_by_css_selector('#collapsible > ytd-metadata-row-renderer').text
            # 비디오의 채널명
            video_info['channel_name'] = driver.find_element_by_css_selector('#channel-name').text
            # 비디오의 구독자 수
            video_info['subscriber_count'] = driver.find_element_by_css_selector('#subscribe-button > ytd-button-renderer').text
            # 비디오의 댓글정보
            video_info['comments']= comments_info(url)
            info.append(video_info)
            driver.close()
        except:
            pass
        
    return info

# Json파일을 생성하는 메소드
def make_json(data,filename):
    with open(filename+'.json','w',encoding='utf-8') as make_file:
        json.dump(data, make_file, ensure_ascii=False, indent='\t')

In [10]:
info = video_info('팝송')

In [17]:
make_json(info,'popsong2')